In [4]:
import requests
import json
import pandas as pd
import time
import os
from dotenv import load_dotenv

In [32]:
load_dotenv()

True

In [59]:
client_id = os.environ.get('CLIENT_ID')
client_secret = os.environ.get('CLIENT_SECRET')

60ksja295in9gxspekqeab33bm0l3x


In [89]:
def get_oauth_token(client_id = client_id, client_secret = client_secret):
    url = 'https://id.twitch.tv/oauth2/token'
    query_string = {'client_id': client_id, 
                    'client_secret': client_secret,
                    'grant_type': 'client_credentials'}
    r = requests.post(url, params=query_string).json()
    return r['access_token']

In [92]:
def get_top_100_categories(client_id = client_id):
    bearer_token = get_oauth_token()
    headers = {'Client-ID': client_id, 
               'Authorization': 'Bearer ' + bearer_token}
    url = r'https://api.twitch.tv/helix/games/top?first=100'
    r = requests.get(url, headers=headers)
    return r

In [109]:
def check_rate_limit_reached(req, ignore_limit = False):
    # checks if there is 1 request before rate limiting
    if int(req.headers['Ratelimit-Remaining']) <= 1:
        # not really preferred could cause trouble
        if ignore_limit:
            return int(req.headers['Ratelimit-Remaining'])
        print('Rate limit refreshes in 30s')
        time.sleep(30)
        print('Ready!')
    # return the remaining tries
    return int(req.headers['Ratelimit-Remaining'])

In [110]:
def get_top_100_streamers_for_each_game(games):
    streamers = {}
    
    bearer_token = get_oauth_token()
    headers = {'Client-ID': client_id, 
               'Authorization': 'Bearer ' + bearer_token}
    url = 'https://api.twitch.tv/helix/streams?first=100&game_id='
    
    for game in games['data']:
        req = requests.get(url + game['id'], headers=headers)
        check_rate_limit_reached(req)
        streamers[game['name']] = json.loads(req.text)
    return streamers

In [112]:
def json_to_df(json):
    total_streams_df = pd.DataFrame(
        columns = ['game_id','id','language','started_at','title','type','user_id','user_name','viewer_count'])
    for game_key in list(json.keys()):
        game_streams_df = pd.json_normalize(json[game_key]['data'])
        total_streams_df = pd.concat([total_streams_df, game_streams_df], sort = False)
    total_streams_df.drop(columns = ['thumbnail_url','tag_ids'], inplace = True)
    return total_streams_df

In [115]:
top_100_games = get_top_100_games().json()
top_streamers = get_top_100_streamers_for_each_game(top_100_games)

In [116]:
df = json_to_df(cock)
df.head(10)

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name
0,509658,42293959406,en,2021-04-12T05:42:43Z,YUGIOH OH OH OH MOVIE NIGHT | !SUBATHON !GIFTC...,live,40934651,ludwig,29735,ludwig,Just Chatting
1,509658,42287932654,en,2021-04-11T18:30:55Z,💦🍑HOT TUB🍑💦shameless e-girl💦-- !instagram: am...,live,125387632,Amouranth,20897,amouranth,Just Chatting
2,509658,42294031710,en,2021-04-12T06:00:16Z,😍😍PRINCESS JACUZZI😍😍 1080P💓IG: indiefoxx.tv💓!s...,live,143917159,Indiefoxx,14772,indiefoxx,Just Chatting
3,509658,41742447996,fr,2021-04-12T08:40:41Z,PONCE - Culture & Plaisir !mdf !prog [20h30 : ...,live,50597026,Ponce,12216,ponce,Just Chatting
4,509658,42294314062,ko,2021-04-12T07:04:01Z,★오늘의뉴스★,live,81857784,쌍베,6070,pacific8815,Just Chatting
5,509658,41399024077,ru,2021-04-12T07:11:48Z,СМОТРИМ 5 ЧАСОВ ОРЛА И РЕШКУ...СКИДКА НА ВОЗРА...,live,188890121,Dmitry_Lixxx,5598,dmitry_lixxx,Just Chatting
6,509658,42294572846,ko,2021-04-12T08:23:32Z,비 싫어'-`,live,148057505,괴물쥐123,4675,tmxk319,Just Chatting
7,509658,41742505084,es,2021-04-12T08:55:07Z,BUENOS DÍAS IN THE MORNING #126,live,501538848,facudiazt,3808,facudiazt,Just Chatting
8,509658,42293816062,zh,2021-04-12T05:11:05Z,💪肥宅改變日記😁【🎁送PS5 or Switch同捆任你選 - 參加輸入 !抽獎】【FW🍊蝦...,live,25202416,蝦愛橘子,3028,shuteye_orange,Just Chatting
9,509658,41742798956,tr,2021-04-12T10:06:37Z,gelirim uğrarım ya | insta : erayozkenar,live,131403189,Eray,2899,eray,Just Chatting


In [117]:
df.tail(10)

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name
0,23857,42294788878,en,2021-04-12T09:41:42Z,shorter stream today! more heart gold :),live,62510206,AriaSaki,1486,ariasaki,Pokémon HeartGold/SoulSilver
1,23857,42293948702,en,2021-04-12T05:40:11Z,NEW HEARTGOLD NUZLOCKE (last save got erased :(),live,29761948,chillindude,329,chillindude,Pokémon HeartGold/SoulSilver
2,23857,41399206989,en,2021-04-12T07:57:57Z,Tough Heartgold Nuzzlock att #4\nGrind Stream ...,live,246233818,FireDolphin60,11,firedolphin60,Pokémon HeartGold/SoulSilver
3,23857,41742412556,de,2021-04-12T08:31:26Z,Pokémon Sacred Gold Hardcore Nuzlocke (!challe...,live,152122408,Scortofy,11,scortofy,Pokémon HeartGold/SoulSilver
4,23857,41742280332,fr,2021-04-12T07:54:45Z,Pokémon HeartGold - Nuzelocke Full Random n°1 ...,live,176010663,Luky_B,11,luky_b,Pokémon HeartGold/SoulSilver
5,23857,41399570509,en,2021-04-12T09:24:09Z,Nuzlocke HeartGold grinding Kanto Badges | !op...,live,446957449,SkyKingSaiyan,8,skykingsaiyan,Pokémon HeartGold/SoulSilver
6,23857,41740981036,en,2021-04-12T01:22:59Z,"Entei, no mic/no cam, 15/19, !quest",live,666569237,RoamingHangar,5,roaminghangar,Pokémon HeartGold/SoulSilver
7,23857,42294590718,en,2021-04-12T08:30:06Z,Ultimate Ironmon SoulSilver,live,430736522,fribbeldip,1,fribbeldip,Pokémon HeartGold/SoulSilver
8,23857,41742791036,fr,2021-04-12T10:04:51Z,Streaming mobile de 123wail,live,499326118,123wail,1,123wail,Pokémon HeartGold/SoulSilver
9,23857,42294829678,en,2021-04-12T09:56:32Z,HG/SS Ultimate IronMON !ironmon,live,38762643,RnR_Train,1,rnr_train,Pokémon HeartGold/SoulSilver


In [118]:
df[df['type'] != 'live'].head()

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name
